<a href="https://colab.research.google.com/github/Taeyeon-Choe/PEgame_1/blob/main/PE_git_test4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi
import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")

!git clone https://github.com/Taeyeon-Choe/PEgame_1.git
%cd PEgame_1

Thu Jul  3 09:19:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install -q stable-baselines3>=2.0.0
!pip install -q gymnasium>=0.28.0
!pip install -q shimmy>=2.0.0
!pip install -q numba>=0.56.0
!pip install -q rich tensorboard

# 설치 확인
import stable_baselines3
import gymnasium
print("패키지 설치 완료!")

패키지 설치 완료!


In [5]:
import sys
sys.path.append('/content/satellite-pursuit-evasion')

# 프로젝트 모듈 임포트
from config.settings import get_config
from environment.pursuit_evasion_env import PursuitEvasionEnv
from training.trainer import create_trainer
from analysis.evaluator import create_evaluator

print("모듈 임포트 성공!")

모듈 임포트 성공!


In [6]:
# 환경 생성 및 테스트
config = get_config(experiment_name="colab_test", debug_mode=True)
env = PursuitEvasionEnv(config)

# 환경 정보 출력
print(f"액션 공간: {env.action_space}")
print(f"관측 공간: {env.observation_space}")

# 랜덤 에피소드 실행
obs = env.reset()
for i in range(10):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    if done:
        print(f"에피소드 종료 (스텝 {i+1}): {info.get('outcome', 'unknown')}")
        break

env.close()

액션 공간: Box(-1.0, 1.0, (3,), float32)
관측 공간: Box(-1.0, 1.0, (9,), float32)


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


In [8]:
# 빠른 학습 설정
config = get_config(experiment_name="colab_quick_training")
config.training.total_timesteps = 10000
config.training.n_envs = 4  # Colab에서는 4개 정도가 적당

# 환경 생성
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

def make_env():
    return PursuitEvasionEnv(config)

# Colab에서는 DummyVecEnv 사용 (더 안정적)
env = DummyVecEnv([make_env for _ in range(config.training.n_envs)])

# 트레이너 생성 및 학습
trainer = create_trainer(env, config)
trainer.setup_model()

print("학습 시작...")
trainer.train()

# 모델 저장
trainer.save_model("/content/quick_model.zip")
print("학습 완료! 모델 저장됨: /content/quick_model.zip")

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


SAC 모델 초기화 중... (Device: cuda)
Using cuda device
Logging to ./logs/sac_training_2025-07-03_09-33-40
학습 시작...
SAC 모델 학습 시작... (총 10,000 스텝)
로그 디렉토리: ./logs/sac_training_2025-07-03_09-33-40


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
# 전체 학습 설정
config = get_config(experiment_name="colab_full_training")
config.training.total_timesteps = 100000
config.training.n_envs = 16
config.training.save_freq = 20000

# 환경 생성
env = DummyVecEnv([make_env for _ in range(config.training.n_envs)])

# 학습 진행 상황 표시를 위한 콜백
from stable_baselines3.common.callbacks import BaseCallback
import matplotlib.pyplot as plt
from IPython.display import clear_output

class ColabProgressCallback(BaseCallback):
    def __init__(self, check_freq=1000):
        super().__init__()
        self.check_freq = check_freq
        self.rewards = []

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            # 진행 상황 출력
            clear_output(wait=True)
            print(f"스텝: {self.n_calls}/{self.locals['total_timesteps']}")
            print(f"진행률: {self.n_calls/self.locals['total_timesteps']*100:.1f}%")

            # 간단한 그래프
            if len(self.rewards) > 0:
                plt.figure(figsize=(10, 4))
                plt.plot(self.rewards)
                plt.title('학습 진행 상황')
                plt.xlabel('체크포인트')
                plt.ylabel('평균 보상')
                plt.grid(True)
                plt.show()

        return True

# 트레이너 생성
trainer = create_trainer(env, config)
trainer.setup_model()

# 콜백 추가하여 학습
progress_callback = ColabProgressCallback(check_freq=5000)
trainer.model.learn(
    total_timesteps=config.training.total_timesteps,
    callback=progress_callback
)

# 모델 저장
trainer.save_model("/content/full_model.zip")
print("전체 학습 완료!")

스텝: 5000/100000
진행률: 5.0%
---------------------------------
| time/              |          |
|    episodes        | 5676     |
|    fps             | 148      |
|    time_elapsed    | 539      |
|    total_timesteps | 80016    |
| train/             |          |
|    actor_loss      | -25      |
|    critic_loss     | 7.39     |
|    ent_coef        | 0.607    |
|    ent_coef_loss   | -2.5     |
|    learning_rate   | 0.0001   |
|    n_updates       | 4994     |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 5680     |
|    fps             | 148      |
|    time_elapsed    | 540      |
|    total_timesteps | 80096    |
| train/             |          |
|    actor_loss      | -25.3    |
|    critic_loss     | 8.16     |
|    ent_coef        | 0.607    |
|    ent_coef_loss   | -2.5     |
|    learning_rate   | 0.0001   |
|    n_updates       | 4999     |
---------------------------------


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


---------------------------------
| time/              |          |
|    episodes        | 5684     |
|    fps             | 148      |
|    time_elapsed    | 540      |
|    total_timesteps | 80144    |
| train/             |          |
|    actor_loss      | -25.2    |
|    critic_loss     | 6.61     |
|    ent_coef        | 0.607    |
|    ent_coef_loss   | -2.5     |
|    learning_rate   | 0.0001   |
|    n_updates       | 5002     |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 5688     |
|    fps             | 148      |
|    time_elapsed    | 541      |
|    total_timesteps | 80192    |
| train/             |          |
|    actor_loss      | -24.7    |
|    critic_loss     | 7.77     |
|    ent_coef        | 0.607    |
|    ent_coef_loss   | -2.49    |
|    learning_rate   | 0.0001   |
|    n_updates       | 5005     |
---------------------------------
---------------------------------
| time/       

/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


---------------------------------
| time/              |          |
|    episodes        | 5696     |
|    fps             | 148      |
|    time_elapsed    | 541      |
|    total_timesteps | 80320    |
| train/             |          |
|    actor_loss      | -25.2    |
|    critic_loss     | 7.99     |
|    ent_coef        | 0.606    |
|    ent_coef_loss   | -2.5     |
|    learning_rate   | 0.0001   |
|    n_updates       | 5013     |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 5700     |
|    fps             | 148      |
|    time_elapsed    | 542      |
|    total_timesteps | 80368    |
| train/             |          |
|    actor_loss      | -24.9    |
|    critic_loss     | 8.19     |
|    ent_coef        | 0.606    |
|    ent_coef_loss   | -2.51    |
|    learning_rate   | 0.0001   |
|    n_updates       | 5016     |
---------------------------------
---------------------------------
| time/       

...

---------------------------------
| time/              |          |
|    episodes        | 7064     |
|    fps             | 147      |
|    time_elapsed    | 674      |
|    total_timesteps | 99568    |
| train/             |          |
|    actor_loss      | -23.8    |
|    critic_loss     | 7.22     |
|    ent_coef        | 0.538    |
|    ent_coef_loss   | -3.09    |
|    learning_rate   | 0.0001   |
|    n_updates       | 6216     |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 7068     |
|    fps             | 147      |
|    time_elapsed    | 675      |
|    total_timesteps | 99648    |
| train/             |          |
|    actor_loss      | -24.1    |
|    critic_loss     | 7.22     |
|    ent_coef        | 0.537    |
|    ent_coef_loss   | -3.11    |
|    learning_rate   | 0.0001   |
|    n_updates       | 6221     |
---------------------------------
---------------------------------
| time/       

/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


---------------------------------
| time/              |          |
|    episodes        | 7076     |
|    fps             | 147      |
|    time_elapsed    | 675      |
|    total_timesteps | 99696    |
| train/             |          |
|    actor_loss      | -23.8    |
|    critic_loss     | 6.24     |
|    ent_coef        | 0.537    |
|    ent_coef_loss   | -3.11    |
|    learning_rate   | 0.0001   |
|    n_updates       | 6224     |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 7080     |
|    fps             | 147      |
|    time_elapsed    | 676      |
|    total_timesteps | 99792    |
| train/             |          |
|    actor_loss      | -24.1    |
|    critic_loss     | 7.42     |
|    ent_coef        | 0.537    |
|    ent_coef_loss   | -3.11    |
|    learning_rate   | 0.0001   |
|    n_updates       | 6230     |
---------------------------------
---------------------------------
| time/       

/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


---------------------------------
| time/              |          |
|    episodes        | 7092     |
|    fps             | 147      |
|    time_elapsed    | 677      |
|    total_timesteps | 99936    |
| train/             |          |
|    actor_loss      | -24      |
|    critic_loss     | 6.87     |
|    ent_coef        | 0.536    |
|    ent_coef_loss   | -3.12    |
|    learning_rate   | 0.0001   |
|    n_updates       | 6239     |
---------------------------------
모델 저장됨: /content/full_model.zip
리플레이 버퍼 저장됨: /content/full_model_replay_buffer.pkl
전체 학습 완료!


In [10]:
from stable_baselines3 import SAC

# 평가용 환경 (단일)
eval_env = PursuitEvasionEnv(config)

# 모델 로드
model = SAC.load("/content/full_model.zip", env=eval_env)

# 평가자 생성
evaluator = create_evaluator(model, eval_env, config)

# 평가 실행
print("모델 평가 중...")
results = evaluator.evaluate_multiple_scenarios(n_tests=20)

# 결과 출력
summary = results['summary']
print(f"\n=== 평가 결과 ===")
print(f"성공률: {summary['success_rate']:.1f}%")
print(f"평균 최종 거리: {summary['avg_final_distance']:.2f} m")
print(f"평균 Delta-V: {summary['avg_evader_delta_v']:.2f} m/s")
print(f"Nash 메트릭: {summary['avg_nash_metric']:.4f}")

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
모델 평가 중...
다중 시나리오 평가 시작... (20 시나리오)
테스트 1/20 실행 중...
테스트 2/20 실행 중...


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


테스트 3/20 실행 중...


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


...

테스트 19/20 실행 중...


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]
/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


테스트 20/20 실행 중...


/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]


평가 결과 저장 완료: ./test_results/2025-07-03_09-46-27

=== 평가 결과 ===
성공률: 100.0%
평균 최종 거리: 5077.68 m
평균 Delta-V: 22.12 m/s
Nash 메트릭: 0.5000


In [13]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 단일 에피소드 실행 및 궤적 수집
obs = eval_env.reset()
states = []
done = False

while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = eval_env.step(action)

    # 물리적 상태 저장
    physical_state = eval_env.state.copy()
    states.append(physical_state[:3])  # 위치만

states = np.array(states)

# 3D 궤적 시각화
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# 궤적 그리기
ax.plot(states[:, 0], states[:, 1], states[:, 2], 'b-', linewidth=2, label='Pursuer Trajectory')
ax.scatter(states[0, 0], states[0, 1], states[0, 2], color='g', s=100, label='Start')
ax.scatter(states[-1, 0], states[-1, 1], states[-1, 2], color='r', s=100, label='End')
ax.scatter(0, 0, 0, color='k', s=150, marker='*', label='Evader (Chief)')

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title(f'위성 추격 궤적 - 결과: {info.get("outcome", "unknown")}')
ax.legend()

plt.show()

# 거리 변화 그래프
distances = np.linalg.norm(states, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(distances, 'b-', linewidth=2)
plt.axhline(y=eval_env.capture_distance, color='r', linestyle='--', label='Capture Distance')
plt.axhline(y=eval_env.evasion_distance, color='g', linestyle='--', label='Evasion Distance')
plt.xlabel('Time Steps')
plt.ylabel('Distance (m)')
plt.title('추격자-회피자 거리 변화')
plt.legend()
plt.grid(True)
plt.show()

/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]
/content/PEgame_1/orbital_mechanics/coordinate_transforms.py:132: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  E = fsolve(kepler_eq, M)[0]
/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 50948 (\N{HANGUL SYLLABLE WI}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 49457 (\N{HANGUL SYLLABLE SEONG}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 52628 (\N{HANGUL SYLLABLE CU}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packa

In [14]:
from google.colab import files

# 모델 다운로드
files.download('/content/full_model.zip')

# 압축 파일로 결과 다운로드
!zip -r results.zip logs/ *.zip
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

	zip warning: name not matched: *.zip
  adding: logs/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-34-01/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-34-01/plots/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-34-01/progress.csv (deflated 58%)
  adding: logs/sac_training_2025-07-03_09-34-01/tensorboard/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-34-01/events.out.tfevents.1751535241.da276e521426.384.1 (deflated 74%)
  adding: logs/sac_training_2025-07-03_09-34-01/models/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-33-40/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-33-40/plots/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-33-40/progress.csv (stored 0%)
  adding: logs/sac_training_2025-07-03_09-33-40/tensorboard/ (stored 0%)
  adding: logs/sac_training_2025-07-03_09-33-40/events.out.tfevents.1751535225.da276e521426.384.0 (deflated 9%)
  adding: logs/sac_training_2025-07-03_09-33-40/models/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>